# Importando Bibliotecas e Direcionando para a shopee

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import pandas as pd

# Crie um objeto Service apontando para o executável do chromedriver
servico = Service(executable_path=r'./chromedriver.exe')

# Crie o navegador Chrome usando o objeto Service
navegador = webdriver.Chrome(service=servico)

# Preencha abaixo o link da pagina da shopee que você precisa raspar os dados
navegador.get('Digite aqui o link')
input('Digite qualquer coisa.')

# Coletando todos os links dos produtos

In [ ]:
time.sleep(3)

links = []  # Lista para armazenar os links
ultima_pagina = None

while True:
    elementos_div = navegador.find_elements(By.CSS_SELECTOR, "div.shop-search-result-view__item.col-xs-2-4")

    for elemento_div in elementos_div:
        elemento_a = elemento_div.find_element(By.CSS_SELECTOR, "a[data-sqe='link']")
        link = elemento_a.get_attribute('href')
        
        # Verifica se o link já foi coletado anteriormente
        if link not in links:
            links.append(link)

    if ultima_pagina is not None and ultima_pagina == links[-1]:
        break

    ultima_pagina = links[-1]

    botao_passar_pagina = navegador.find_elements(By.CSS_SELECTOR, "button.shopee-icon-button.shopee-icon-button--right")

    navegador.execute_script("arguments[0].click();", botao_passar_pagina[0])
    time.sleep(6)  # Aguardar 6 segundos para o conteúdo carregar na próxima página

# Coletando os dados pagina a pagina pelo link.

In [ ]:
lista_de_info = []

def encontrar_elemento(selector, default="Não exibe"):
    try:
        return navegador.find_element(By.CSS_SELECTOR, selector).text
    except NoSuchElementException:
        return default
    
numero_de_repeticao = 0

for link in links:
    numero_de_repeticao += 1
    navegador.get(link)
    time.sleep(7)  # Aguardar 7 segundos antes de localizar os elementos

    produto = encontrar_elemento("[class='pqTWkA']")
    nome = encontrar_elemento("div._44qnta span")
    vendas = encontrar_elemento("[class='e9sAa2']")
    avaliacoes = encontrar_elemento("[class='_1k47d8']", 0)
    estrelas = encontrar_elemento("[class='_1k47d8 _046PXf']", "Não exibe")

    dicionario = {'nome_do_produto': nome, 'preço': produto, 'vendas': vendas, 'avaliacoes': avaliacoes, 'estrelas': estrelas, 'link': link}
    print(numero_de_repeticao,dicionario)
    lista_de_info.append(dicionario)

navegador.quit()

# Tratamento de dados com o Pandas e gerando arquivo xlsx.

In [ ]:
for dicionario in lista_de_info:
    df = pd.DataFrame.from_dict(dicionario, orient='index', columns=['valor'])

compilada = None  # Inicializa compilada como None

for dicionario in lista_de_info:
    df = pd.DataFrame.from_dict(dicionario, orient='index', columns=['valor'])
    df_invertido = df.transpose()  # ou df.T
    if compilada is None:
        compilada = df_invertido  # Se compilada é None, atribui o primeiro DataFrame diretamente
    else:
        lista_tabelas = [compilada, df_invertido]
        compilada = pd.concat(lista_tabelas)  # Concatena os DataFrames
        
compilada.to_excel('Produtos.xlsx',index = False)
